In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split

In [3]:
#
# Load config from 'config.txt'
#
def get_config():
    #
    import sys
    import yaml
    #
    with open("config.txt", "r") as f:
        configtxt = f.readlines()
    configtxt = "".join(configtxt)
    return yaml.load(configtxt, Loader=yaml.FullLoader)

In [4]:
def lag_time_series(serie, Pmax, remove_lag_zero=True):
    df = pd.DataFrame(serie)
    for p in range(1, Pmax + 1):
        df["P_{:d}".format(p)] = serie.shift(periods=p, fill_value=0)
    if remove_lag_zero:
        df.pop(df.columns[0])
    return df

In [18]:
from operator import itemgetter

class ModeloONI:
    def __init__(self, P=3):
        self.ONI = pd.read_csv("data/data.csv").ONI[0:240]
        self.P = P

    def build_table(self, Pmax):

        for p in range(1, Pmax + 1):
            x = lag_time_series(serie=self.ONI, Pmax=p)
            y = self.ONI
            (
                x_train,
                x_test,
                y_train_real,
                y_test_real,
            ) = train_test_split(x, y, test_size=0.4, random_state=0)

            m = linear_model.LinearRegression()
            m.fit(x_train, y_train_real)

            y_train_predict = m.predict(x_train)
            y_test_predict = m.predict(x_test)
            mse_train = metrics.mean_squared_error(y_train_real, y_train_predict)
            mse_test = metrics.mean_squared_error(y_test_real, y_test_predict)
            print("{:2d}  {:6.4f}  {:6.4f}".format(p, mse_train, mse_test))

    def fit_to_all_data(self):
        #
        # Entrena el modelo usando todos los datos historicos disponibles
        #
        x = lag_time_series(serie=self.ONI, Pmax=self.P)
        y = self.ONI
        self.m = linear_model.LinearRegression()
        self.m.fit(x, y)
        self.errors = (self.ONI - self.m.predict(x))[self.P + 1 :]
        self.errors = self.errors.tolist()

    def get_syntetic_series(self, t_base):
        y = self.ONI.copy()
        for t in range(t_base, 264):
            y[t] = self.m.intercept_
            for p in range(1, self.P + 1):
                y[t] += y[t - p] * self.m.coef_[p - 1]
            y[t] += self.errors[np.random.randint(0, len(self.errors))]
        return y
        
    def run(self):

        self.fit_to_all_data()
        
        CONFIG = get_config()

        for t_base in range(204, 241):

            #
            # Genera las simulaciones
            #
            simulations = {}
            for i_sim in range(CONFIG["ONI"]["NUMSIM"]):
                simulations[i_sim] = self.get_syntetic_series(t_base)

            #
            # Computa la media
            #
            means = []
            for i_sim in range(CONFIG["ONI"]["NUMSIM"]):
                means.append((i_sim, simulations[i_sim][t_base : t_base + 24].mean()))

            #
            # ordena por las medias
            #
            means = sorted(means, key=itemgetter(1))

            #
            # Percentiles
            #
            escenarios = CONFIG["ONI"]["ESCENARIOS"].copy()
            escenarios = [int(len(means) * float(e) / 100) for e in escenarios]
            escenarios = [means[e][0] for e in escenarios]

            for e, p in zip(escenarios, CONFIG["ONI"]["ESCENARIOS"].copy()):
                filename = str(t_base) + "/" + "ONI_" + str(p) + ".csv"
                simulations[e].to_csv(filename, index=False)

ModeloONI().run()

In [20]:
class ModeloAportes:
    def __init__(self, P_ONI, P_Aportes):
        x = pd.read_csv("data/data.csv")
        self.ONI = x.ONI[0:240]
        self.Mes = x.Mes[0:240]
        self.Aportes = x.Aportes_GWh[0:240]
        self.dummies = x.loc[
            0:239, ["PRJ_0", "PRJ_1", "PRJ_2", "PRJ_3", "PRJ_4", "PRJ_5", "PRJ_6"]
        ]
        self.P_ONI = P_ONI
        self.P_Aportes = P_Aportes

        
    def step_1_remove_ONI(self):
        #
        # Remueve el efecto del ONI de todos los datos
        #
        x = lag_time_series(serie=self.ONI, Pmax=self.P_ONI, remove_lag_zero=False)
        x = pd.concat([x, self.dummies], axis=1)
        y = self.Aportes
        self.modelo_ONI = linear_model.LinearRegression()
        self.modelo_ONI.fit(x, y)
        self.Aportes_sin_ONI = self.Aportes - self.modelo_ONI.predict(x)

        
    def step_2_remove_seasonal_cycle(self):
        #
        x = pd.concat(
            [pd.DataFrame(self.Mes), pd.DataFrame(self.Aportes_sin_ONI)], axis=1
        )
        x.columns = [x.columns[0], "Aportes_sin_ONI"]
        self.means_stds = x.groupby("Mes").agg({"Aportes_sin_ONI": [np.mean, np.std]})
        self.means_stds.columns = ["mean_mes", "std_mes"]
        self.Aportes_sin_ONI_z = [
            (v - self.means_stds["mean_mes"][t]) / self.means_stds["std_mes"][t]
            for v, t in zip(self.Aportes_sin_ONI, self.Mes)
        ]
        self.Aportes_sin_ONI_z = pd.Series(self.Aportes_sin_ONI_z)
        
        
    def step_3_fit(self):
        #
        x = lag_time_series(serie=self.Aportes_sin_ONI_z, Pmax=self.P_Aportes)
        self.modelo_Aportes = linear_model.LinearRegression()
        self.modelo_Aportes.fit(x, self.Aportes_sin_ONI_z)
        self.errors = (self.Aportes_sin_ONI_z - self.modelo_Aportes.predict(x))[self.P_Aportes + 1 :]
        self.errors = self.errors.tolist()


    def step_4_generate_syntetic_Aportes_sin_ONI_z(self, t_base):
        #
        y = self.Aportes_sin_ONI_z.copy()

        for t in range(t_base, 264):
            y[t] = self.modelo_Aportes.intercept_
            for p in range(1, self.P_Aportes + 1):
                y[t] += y[t - p] * self.modelo_Aportes.coef_[p - 1]
            y[t] += self.errors[np.random.randint(0, len(self.errors))]
        self.sintetic_Aportes_sin_ONI_z = y.copy()

    def step_5_generate_syntetic_Aportes_sin_ONI(self):

        x = pd.DataFrame({"z": self.sintetic_Aportes_sin_ONI_z})
        x["Mes"] = 0
        mes = 1
        for t in range(len(x.Mes)):
            x.loc[t, "Mes"] = mes
            mes = mes + 1
            mes = mes if mes <= 12 else 1

        x["Mean"] = x.Mes.map(lambda w: self.means_stds.mean_mes[w])
        x["Std"] = x.Mes.map(lambda w: self.means_stds.std_mes[w])
        x["sintetic_Aportes_sin_ONI"] = self.sintetic_Aportes_sin_ONI_z * x.Std + x.Mean
        self.sintetic_Aportes_sin_ONI = x.sintetic_Aportes_sin_ONI
        
        
    def step_6_generate_syntetic_Aportes(self, sintetica_ONI, t_base):

        dummies = pd.DataFrame(
            1, columns=self.dummies.columns, index=range(240, 240 + 24)
        )
        dummies = pd.concat([self.dummies, dummies], axis=0)
        

        x = lag_time_series(serie=sintetica_ONI, Pmax=self.P_ONI, remove_lag_zero=False)
        
        x = pd.concat([x, dummies], axis=1)
        
        self.sintetic_Aportes = (
            self.sintetic_Aportes_sin_ONI + self.modelo_ONI.predict(x)
        )
        

        
    def run(self):
        
        CONFIG = get_config()

        self.step_1_remove_ONI()
        self.step_2_remove_seasonal_cycle()
        self.step_3_fit()
    
        for t_base in range(204, 241):

            self.step_4_generate_syntetic_Aportes_sin_ONI_z(t_base)
            self.step_5_generate_syntetic_Aportes_sin_ONI()

            for escenario in CONFIG["ONI"]["ESCENARIOS"]:
            
                for i_serie in range(CONFIG["APORTES"]["NUMSIM"]):
            
                    filename = str(t_base) + "/" + "ONI_" + str(escenario) + ".csv"
                    sintetica_ONI = pd.read_csv(filename)['ONI']
                    self.step_6_generate_syntetic_Aportes(sintetica_ONI, t_base)
                    self.sintetic_Aportes.to_csv("{}/APORTES_{:04d}_ONI_{}.csv".format(t_base, i_serie, escenario), index=False)

        
    def remove_ONI_table(self, Pmax):
        #
        # Paso 1
        #
        for p in range(0, Pmax + 1):
            x = lag_time_series(serie=self.ONI, Pmax=p, remove_lag_zero=False)
            x = pd.concat([x, self.dummies], axis=1)
            y = self.Aportes
            (
                x_train,
                x_test,
                y_train_real,
                y_test_real,
            ) = train_test_split(x, y, test_size=0.4, random_state=0)

            m = linear_model.LinearRegression()
            m.fit(x_train, y_train_real)

            y_train_predict = m.predict(x_train)
            y_test_predict = m.predict(x_test)
            mse_train = metrics.mean_squared_error(y_train_real, y_train_predict)
            mse_test = metrics.mean_squared_error(y_test_real, y_test_predict)
            print("{:2d}  {:6.4f}  {:6.4f}".format(p, mse_train, mse_test))



    def model_Aportes_sin_ONI_z_table(self, Pmax):
        #
        # Paso 4
        #
        for p in range(1, Pmax + 1):
            x = lag_time_series(
                serie=self.Aportes_sin_ONI_z, Pmax=p, remove_lag_zero=True
            )
            y = self.Aportes_sin_ONI_z

            (
                x_train,
                x_test,
                y_train_real,
                y_test_real,
            ) = train_test_split(x, y, test_size=0.4, random_state=0)

            m = linear_model.LinearRegression()
            m.fit(x_train, y_train_real)

            y_train_predict = m.predict(x_train)
            y_test_predict = m.predict(x_test)
            mse_train = metrics.mean_squared_error(y_train_real, y_train_predict)
            mse_test = metrics.mean_squared_error(y_test_real, y_test_predict)
            print("{:2d}  {:14.4f}  {:14.4f}".format(p, mse_train, mse_test))




ModeloAportes(P_ONI=3, P_Aportes = 14).run()

In [ ]:
for i in range(204, 241):
    !mkdir {i}

In [ ]:
pd.DataFrame(1, columns=["A", "B"], index=["a", "b", "c"])

In [ ]:
"{:04d}".format(10)

In [ ]:
import matplotlib.pyplot as plt

data = pd.read_csv("data/data.csv")
plt.plot(data.ONI)

In [ ]:
%%writefile config.txt
ONI: [10, 50, 90]

In [ ]:
import yaml

In [ ]:
%%writefile config.txt
ONI: 
    NUMSIM: 100
    ESCENARIOS: [10, 50, 90]

In [ ]:
class ModeloAportes:
    def __init__(self):
        x = pd.read_csv("data/data.csv")
        self.ONI = x.ONI[0:240]
        self.Mes = x.Mes[0:240]
        self.Aportes = x.Aportes_GWh[0:240]
        self.dummies = x.loc[
            0:239, ["PRJ_0", "PRJ_1", "PRJ_2", "PRJ_3", "PRJ_4", "PRJ_5", "PRJ_6"]
        ]

    def remove_ONI_table(self, Pmax):

        #
        # Paso 1
        #
        for p in range(0, Pmax + 1):
            x = lag_time_series(serie=self.ONI, Pmax=p, remove_lag_zero=False)
            x = pd.concat([x, self.dummies], axis=1)
            y = self.Aportes
            (
                x_train,
                x_test,
                y_train_real,
                y_test_real,
            ) = train_test_split(x, y, test_size=0.4, random_state=0)

            m = linear_model.LinearRegression()
            m.fit(x_train, y_train_real)

            y_train_predict = m.predict(x_train)
            y_test_predict = m.predict(x_test)
            mse_train = metrics.mean_squared_error(y_train_real, y_train_predict)
            mse_test = metrics.mean_squared_error(y_test_real, y_test_predict)
            print("{:2d}  {:6.4f}  {:6.4f}".format(p, mse_train, mse_test))

    def remove_ONI(self, p):
        #
        # Paso 2
        #
        x = lag_time_series(serie=self.ONI, Pmax=p, remove_lag_zero=False)
        x = pd.concat([x, self.dummies], axis=1)
        y = self.Aportes
        m = linear_model.LinearRegression()
        m.fit(x, y)
        self.Aportes_sin_ONI = self.Aportes - m.predict(x)

        self.model_ONI = m

        #  plt.plot(self.Aportes, color='blue')
        #  plt.plot(m.predict(x), color='black')
        #  plt.plot(self.Aportes_sin_ONI, color='k')

    def remove_seasonal_cycle(self):
        #
        # Paso 3
        #
        x = pd.concat(
            [pd.DataFrame(self.Mes), pd.DataFrame(self.Aportes_sin_ONI)], axis=1
        )
        x.columns = [x.columns[0], "Aportes_sin_ONI"]
        self.means_stds = x.groupby("Mes").agg({"Aportes_sin_ONI": [np.mean, np.std]})
        self.means_stds.columns = ["mean_mes", "std_mes"]

        self.Aportes_sin_ONI_z = [
            (v - self.means_stds["mean_mes"][t]) / self.means_stds["std_mes"][t]
            for v, t in zip(self.Aportes_sin_ONI, self.Mes)
        ]
        self.Aportes_sin_ONI_z = pd.Series(self.Aportes_sin_ONI_z)

        # plt.plot(self.Aportes_sin_ONI_z)

    def model_Aportes_sin_ONI_z_table(self, Pmax):
        #
        # Paso 4
        #
        for p in range(1, Pmax + 1):
            x = lag_time_series(
                serie=self.Aportes_sin_ONI_z, Pmax=p, remove_lag_zero=True
            )
            y = self.Aportes_sin_ONI_z

            (
                x_train,
                x_test,
                y_train_real,
                y_test_real,
            ) = train_test_split(x, y, test_size=0.4, random_state=0)

            m = linear_model.LinearRegression()
            m.fit(x_train, y_train_real)

            y_train_predict = m.predict(x_train)
            y_test_predict = m.predict(x_test)
            mse_train = metrics.mean_squared_error(y_train_real, y_train_predict)
            mse_test = metrics.mean_squared_error(y_test_real, y_test_predict)
            print("{:2d}  {:14.4f}  {:14.4f}".format(p, mse_train, mse_test))

    def generate_sintetic_Aportes_sin_ONI_z(self, P):
        x = lag_time_series(serie=self.Aportes_sin_ONI_z, Pmax=P)
        m = linear_model.LinearRegression()
        m.fit(x, self.Aportes_sin_ONI_z)
        errors = (self.Aportes_sin_ONI_z - m.predict(x))[P + 1 :]
        errors = errors.tolist()
        y = self.Aportes_sin_ONI_z.tolist()
        y.extend([0] * 204)

        for t in range(240, len(y)):
            y[t] = m.intercept_
            for p in range(1, P + 1):
                y[t] += y[t - p] * m.coef_[p - 1]
            y[t] += errors[np.random.randint(0, len(errors))]

        self.sintetic_Aportes_sin_ONI_z = y

        #  plt.plot(y)
        #  plt.axvline(x=240, c='red')
        # return y

    def generate_sintetic_Aportes_sin_ONI(self):

        x = pd.DataFrame({"z": self.sintetic_Aportes_sin_ONI_z})
        x["Mes"] = 0
        mes = 1
        for t in range(len(x.Mes)):
            x.loc[t, "Mes"] = mes
            mes = mes + 1
            mes = mes if mes <= 12 else 1

        x["Mean"] = x.Mes.map(lambda w: self.means_stds.mean_mes[w])
        x["Std"] = x.Mes.map(lambda w: self.means_stds.std_mes[w])

        x["sintetic_Aportes_sin_ONI"] = self.sintetic_Aportes_sin_ONI_z * x.Std + x.Mean

        self.sintetic_Aportes_sin_ONI = x.sintetic_Aportes_sin_ONI

        #  plt.plot(self.sintetic_Aportes_sin_ONI)
        # plt.axvline(x=240, c='red')

    def generate_sintetic_Aportes(self):

        #  sintetic_Aportes = self.sintetic_Aportes_sin_ONI

        sintetica_ONI = pd.Series(ModeloONI().generate_sintetic(P=3))

        x = lag_time_series(serie=sintetica_ONI, Pmax=3, remove_lag_zero=False)
        dummies = pd.DataFrame(
            1, columns=self.dummies.columns, index=range(240, 240 + 204)
        )
        dummies = pd.concat([self.dummies, dummies], axis=0)
        x = pd.concat([x, dummies], axis=1)

        #

        self.sintetic_Aportes = self.sintetic_Aportes_sin_ONI + self.model_ONI.predict(
            x
        )

        plt.plot(self.sintetic_Aportes)

        #  return self.sintetic_Aportes
        #  print(x)
        #  print(self.model_ONI.predict(x) )


plt.figure(figsize=(14, 5))
#  ModeloAportes().remove_ONI_table(12)
m = ModeloAportes()
m.remove_ONI(p=3)
m.remove_seasonal_cycle()
#  m.model_Aportes_sin_ONI_z_table(Pmax=30)
m.generate_sintetic_Aportes_sin_ONI_z(14)
m.generate_sintetic_Aportes_sin_ONI()
m.generate_sintetic_Aportes()